<a href="https://colab.research.google.com/github/jufucu/Tarea2/blob/main/Sqlite3_Codigos_de_Ejemplo_programming_python_pag_1332.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Programming Python: Powerful Object-Oriented Programming Fourth Edición](http://index-of.es/Python/OReilly.Programming.Python.4.Edition.Dec.2010.pdf)

**An SQL Database API Tutorial with SQLite**. PAGINA 1332

In [ ]:
import sqlite3
import os
import sys

In [ ]:
# Se conecta a la base de datos, si no existe la crea
conn = sqlite3.connect("dbase1.sqlite3")

In [ ]:
"""
Making databases and tables
Next, let’s make a cursor for submitting SQL statements to the database server, and
submit one to create a first table:
"""
curs = conn.cursor()
sql = """
CREATE TABLE IF NOT EXISTS people (name char(30),
                    job char(10),
                    pay int(4))
"""
curs.execute(sql)

**Adding records**

So far, we’ve logged in (which just means opening a local file in SQLite) and created a
table. Next let’s start a new Python session and create some records. There are three
basic statement-based approaches we can use here: inserting one row at a time or inserting multiple rows with a single call statement or a Python loop. Here is the simple case (I’m omitting some call return values here if they are irrelevant to the story):

In [ ]:
import sqlite3

conn = sqlite3.connect("dbase1.sqlite3")

curs = conn.cursor()

curs.execute('INSERT INTO people VALUES(?,?,?)', ('Bob', 'dev', 5000))

In [ ]:
curs.rowcount

1

In [ ]:
curs.executemany('insert into people values (?, ?, ?)',
                 [ ('Sue', 'mus', '70000'),
                  ('Ann', 'mus', '60000')])

In [ ]:
curs.rowcount

2

In [ ]:
rows = [['Tom', 'mgr', 100000],
        ['Kim', 'adm', 30000],
        ['pat', 'dev', 90000]]

for row in rows:
  curs.execute('insert into people values (? , ?, ?)', row)

In [ ]:
conn.commit()

Blending Python and SQL like this starts to open up all sorts of interesting possibilities.

Notice the last command; we always need to call the connection’s commit method to
write our changes out to the database. Otherwise, when the connection is closed, our changes may be lost. In fact, until we call the commit method, none of our inserts may be visible from other database connections.

Technically, the API suggests that a connection object should automatically call its rollback method to back out changes that have not yet been committed, when it is
closed (which happens manually when its close method is called, or automatically
when the connection object is about to be garbage collected). For database systems
that don’t support transaction commit and rollback operations, these calls may do
nothing. SQLite implements both the commit and rollback methods; the latter rolls
back any changes made since the last commit .

# Running queries
OK, we’ve now added six records to our database table. Let’s run an SQL query to see how we did:

In [ ]:
curs.execute('select * from people')
curs.fetchall()

[('Bob', 'dev', 5000),
 ('Sue', 'mus', 70000),
 ('Ann', 'mus', 60000),
 ('Tom', 'mgr', 100000),
 ('Kim', 'adm', 30000),
 ('pat', 'dev', 90000)]

Run an SQL select statement with a cursor object to grab all rows and call the cursor’s fetchall to retrieve them. They come back to our script as a sequence of sequences. In this module, it’s a list of tuples—the outer list represents the result table, the nested tuples are that table’s rows, and the nested tuple’s contents are the column data. Because it’s all Python data, once we get the query result, we process it with normal Python code. For example, to make the display a bit more coherent, loop through the query’s
result as usual:

In [ ]:
curs.execute('select * from people')
for row in curs.fetchall():
  print(row)

('Bob', 'dev', 5000)
('Sue', 'mus', 70000)
('Ann', 'mus', 60000)
('Tom', 'mgr', 100000)
('Kim', 'adm', 30000)
('pat', 'dev', 90000)


Tuple unpacking comes in handy in loops here, too, to pick out column values as we
go. Here’s a simple formatted display of two of the columns’ values:

In [ ]:
curs.execute('select * from people')
for (name, job, pay) in curs.fetchall():
    print(name, ':', pay)

Bob : 5000
Sue : 70000
Ann : 60000
Tom : 100000
Kim : 30000
pat : 90000


Because the query result is a sequence, we can use Python’s powerful sequence and iteration tools to process it. For instance, to select just the name column values, we can run a more specific SQL query and get a list of tuples:

In [ ]:
curs.execute('select name from people')
names = curs.fetchall()
names

[('Bob',), ('Sue',), ('Ann',), ('Tom',), ('Kim',), ('pat',)]

Or we can use a Python list comprehension to pick out the fields we want—by using
Python code, we have more control over the data’s content and format:

In [ ]:
curs.execute('select * from people')
names = [rec[0] for rec in curs.fetchall()]
names

['Bob', 'Sue', 'Ann', 'Tom', 'Kim', 'pat']

The fetchall call we’ve used so far fetches the entire query result table all at once, as a single sequence (an empty sequence comes back, if the result is empty). That’s convenient, but it may be slow enough to block the caller temporarily for large result tables or generate substantial network traffic if the server is running remotely (something could easily require a parallel thread in GUI). To avoid such a bottleneck, we can also grab just one row, or a bunch of rows, at a time with fetchone and fetchmany . The fetchone call returns the next result row or a None false value at the end of the table:

In [ ]:
curs.execute('select * from people')
while True:
    row = curs.fetchone()
    if not row:
        break
    print(row)

('Bob', 'dev', 5000)
('Sue', 'mus', 70000)
('Ann', 'mus', 60000)
('Tom', 'mgr', 100000)
('Kim', 'adm', 30000)
('pat', 'dev', 90000)


The fetchmany call returns a sequence of rows from the result, but not the entire table; you can specify how many rows to grab each time with a parameter or rely on the default as given by the cursor’s arraysize attribute. Each call gets at most that many more rows from the result or an empty sequence at the end of the table:

In [ ]:
curs.execute('select * from people')
while True:
    rows = curs.fetchmany() # size=N optional argument
    if not rows: break
    for row in rows:
        print(row)

('Bob', 'dev', 5000)
('Sue', 'mus', 70000)
('Ann', 'mus', 60000)
('Tom', 'mgr', 100000)
('Kim', 'adm', 30000)
('pat', 'dev', 90000)


For this module at least, the result table is exhausted after a fetchone or fetchmany
returns a False value. The DB API says that fetchall returns “all (remaining) rows,”
so you generally need to call execute again to regenerate results before fetching
new data:

In [ ]:
curs.fetchone()

In [ ]:
curs.fetchmany()

[]

In [ ]:
curs.fetchall()

[]

Naturally, we can do more than fetch an entire table; the full power of the SQL language is at your disposal in Python:

In [ ]:
curs.execute('select name, job from people where pay > 60000')
curs.fetchall()

[('Sue', 'mus'), ('Tom', 'mgr'), ('pat', 'dev')]

The last query fetches name and job fields for people who earn more than $60,000. The
next is similar, but passes in the selection value as a parameter and orders the result
table:

In [ ]:
query = 'select name, job from people where pay >= ? order by name'
curs.execute(query, [60000])
for row in curs.fetchall():
    print(row)

('Ann', 'mus')
('Sue', 'mus')
('Tom', 'mgr')
('pat', 'dev')


# **Running updates**

Cursor objects also are used to submit SQL update statements to the database server updates, deletes, and inserts. We’ve already seen the insert statement at work. Let’s start a new session to perform some other kinds of updates; we begin with the same data we had in the prior session:


quede en la pagina 1338